In [11]:
# Import / install relevant Python packages
import numpy as np
import pandas as pd  
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.cluster import (KMeans, MiniBatchKMeans, AgglomerativeClustering, DBSCAN)
from sklearn.neighbors import NearestNeighbors

In [12]:
alpha = 50 #percentile - default is 50 - mediane
# Define the key inputs for the K-means model
#n_clusters = 5
n_max_iter = 1000
n_init     = 10
init_type  = 'k-means++'
#number of clusters list 
K = [5,10,50, 100, 500, 1000,1500]


In [3]:
feature_names_list = ['mom1m','absacc','acc','aeavol','age','agr','baspread',
'beta','betasq','bm','bm_ia','cash','cashdebt','cashpr','cfp','cfp_ia',
'chatoia','chcsho','chempia','chinv','chmom','chpmia','chtx','cinvest',
'convind','currat','depr','divi','divo','dolvol','dy','ear','egr','ep',
'gma','herf','hire','idiovol','ill','indmom','invest','lev','lgr','maxret',
'ms','mve_ia','mvel1','nincr','operprof','pchcapx_ia','pchcurrat','pchdepr',
'pchgm_pchsale','pchquick','pchsale_pchrect','pctacc','pricedelay',
'ps','quick','rd','retvol','roaq','roeq','roic','rsup','salecash',
'salerec','securedind','sgr','sin','sp','std_dolvol','std_turn',
'tang','tb','turn','zerotrade']

mom_names_list = ["mom1m","mom2m","mom3m","mom4m","mom5m","mom6m","mom7m","mom8m","mom9m","mom10m","mom11m","mom12m",
                "mom13m","mom14m","mom15m","mom16m","mom17m","mom18m","mom19m","mom20m","mom21m","mom22m","mom23m","mom24m",
                "mom25m","mom26m","mom27m","mom28m","mom29m","mom30m","mom31m","mom32m","mom33m","mom34m","mom35m","mom36m",
                "mom37m","mom38m","mom39m","mom40m","mom41m","mom42m","mom43m","mom44m","mom45m","mom46m","mom47m","mom48m"]

feature_names_list_full =  feature_names_list + mom_names_list[1:]

PCA_components_number = len(feature_names_list_full)
PCA_components_column_names = ['PCA '+ str(i+1) for i in range(PCA_components_number)]
PCA_components_tresholds_column_names=['n_to_reach_90','n_to_reach_95','n_to_reach_99']
Original_df_columns_names = ['DATE','permno','sic2']
PCA_table_full_columns_names = Original_df_columns_names+PCA_components_column_names+PCA_components_tresholds_column_names
KMeans_column_names = ['cluster k=5','distance k=5','outlier k=5','cluster k=10','distance k=10','outlier k=10','cluster k=50','distance k=50','outlier k=50',
                        'cluster k=100','distance k=100','outlier k=100','cluster k=500','distance k=500','outlier k=500','cluster k=1000','distance k=1000','outlier k=1000',
                        'cluster k=1500','distance k=1500','outlier k=1500']


In [4]:
df_PCA = pd.read_csv('D:/USB Drive/Data/DachengXiu/df_PCA_done.csv')

In [6]:
def distance_of_percentile_to_nearest_neighbors(df_for_determine_percentile_knn,alpha = 50):
    k = 2
    # initialize model
    neigh = NearestNeighbors(n_neighbors=k, n_jobs=-1)
    # train for getting nearest neighbour
    neigh.fit(df_for_determine_percentile_knn)
    dist, ind = neigh.kneighbors(df_for_determine_percentile_knn)
    return np.percentile(dist[:,1], alpha)


In [7]:
def create_KMeans_model(df_for_determine_labels,n_clusters):
    # Create an instance of the class KMeans, fit the model, and predict cluster memberships
    kmeans_model = KMeans(n_clusters=n_clusters,
                        init=init_type,
                        n_init=n_init,
                        max_iter=n_max_iter,
                        random_state=1)
    
    kmeans = kmeans_model.fit(df_for_determine_labels)
    return kmeans

In [9]:
def calculate_distances_to_centroide_of_belonging_cluster(kmeans,df_for_determine_labels,n_clusters):
    lebels = kmeans.labels_
    distances_to_centroides = kmeans.transform(df_for_determine_labels)
    distance_to_belonging_cluster_centroide = np.array([distances_to_centroides[i][x] for i,x in enumerate(lebels)])
    return distance_to_belonging_cluster_centroide

In [10]:
#inicilize df for KMeans - make empty df
#initial dataframe
df_KMeans=pd.DataFrame(columns=Original_df_columns_names + KMeans_column_names)
dates = sorted(df_PCA['DATE'].unique())
for date in dates:
    print("DATE ",date)
    df_KMeans_window = df_PCA[df_PCA['DATE'] == date].sort_values('permno')

    #determen list of column names of PCA components needed to explain 99% of variance
    number_of_PCA_components_for_99_variance_explainability =  df_KMeans_window['n_to_reach_99'].min() #workaround to get single value
    PCA_components_for_99_variance_explainability_column_names = ['PCA '+ str(i+1) for i in range(number_of_PCA_components_for_99_variance_explainability)]
    
    #calculate treshold for all points in data set - needed for outlier detection
    #I have some phyton variable type bug ( TypeError: 'numpy.float64' object is not callable ) and I can not understand it. I will put code of function in here
    #distance_of_percentile_to_nearest_neighbors = distance_of_percentile_to_nearest_neighbors(df_KMeans_window[PCA_components_for_99_variance_explainability_column_names],alpha)
    #code of function distance_of_percentile_to_nearest_neighbors
    r = 2 #I will change k into r since there is k already in use
    # initialize model
    neigh = NearestNeighbors(n_neighbors=r, n_jobs=-1)
    # train for getting nearest neighbour
    neigh.fit(df_KMeans_window[PCA_components_for_99_variance_explainability_column_names])
    dist, ind = neigh.kneighbors(df_KMeans_window[PCA_components_for_99_variance_explainability_column_names])
    distance_of_percentile_to_nearest_neighbors = np.percentile(dist[:,1], alpha)
    
    #Use number of clusters smaler then number of data points 
    list_of_cluster_numbers = [x for x in K if x < len(df_KMeans_window)]
    for k in list_of_cluster_numbers:
        #Create KMeans model 
        kmeans_fited = create_KMeans_model(df_KMeans_window[PCA_components_for_99_variance_explainability_column_names],k)
        #determine cluster lables and fill data to df
        df_KMeans_window[f'cluster k={k}'] = kmeans_fited.labels_
        #calculate distances to centroide of belonging_cluster for every data point
        df_KMeans_window[f'distance k={k}'] = calculate_distances_to_centroide_of_belonging_cluster(kmeans_fited,df_KMeans_window[PCA_components_for_99_variance_explainability_column_names],k)
        #setting flag 1 if it is outlier
        df_KMeans_window[f'outlier k={k}'] = 0
        df_KMeans_window[f'outlier k={k}'].values[df_KMeans_window[f'distance k={k}'] > distance_of_percentile_to_nearest_neighbors] = 1
    
    df_KMeans=pd.concat([df_KMeans,df_KMeans_window],axis=0)
df_KMeans = df_KMeans[Original_df_columns_names+KMeans_column_names]


DATE  19791231
DATE  19800131
DATE  19800229
DATE  19800331
DATE  19800430
DATE  19800530
DATE  19800630
DATE  19800731
DATE  19800829
DATE  19800930
DATE  19801031
DATE  19801128
DATE  19801231
DATE  19810130
DATE  19810227
DATE  19810331
DATE  19810430
DATE  19810529
DATE  19810630
DATE  19810731
DATE  19810831
DATE  19810930
DATE  19811030
DATE  19811130
DATE  19811231
DATE  19820129
DATE  19820226
DATE  19820331
DATE  19820430
DATE  19820528
DATE  19820630
DATE  19820730
DATE  19820831
DATE  19820930
DATE  19821029
DATE  19821130
DATE  19821231
DATE  19830131
DATE  19830228
DATE  19830331
DATE  19830429
DATE  19830531
DATE  19830630
DATE  19830729
DATE  19830831
DATE  19830930
DATE  19831031
DATE  19831130
DATE  19831230
DATE  19840131
DATE  19840229
DATE  19840330
DATE  19840430
DATE  19840531
DATE  19840629
DATE  19840731
DATE  19840831
DATE  19840928
DATE  19841031
DATE  19841130
DATE  19841231
DATE  19850131
DATE  19850228
DATE  19850329
DATE  19850430
DATE  19850531
DATE  1985

In [5]:
#df_KMeans.to_csv('D:/USB Drive/Data/DachengXiu/df_KMeans.csv')

Analysis:

In [13]:
df_KMeans = pd.read_csv('D:/USB Drive/Data/DachengXiu/df_KMeans.csv')

In [14]:
df_1990_2000 = df_KMeans[(df_KMeans['DATE'] > 19891231 ) & (df_KMeans['DATE'] < 20210101)]
outliers_column_names_list =  ['outlier k=5','outlier k=10','outlier k=50','outlier k=100','outlier k=500','outlier k=1000','outlier k=1500']
clusters_column_names_list =  ['cluster k=5','cluster k=10','cluster k=50','cluster k=100','cluster k=500','cluster k=1000','cluster k=1500']

In [6]:
df_1990_2000

,Unnamed: 0,DATE,permno,sic2,cluster k=5,distance k=5,outlier k=5,cluster k=10,distance k=10,outlier k=10,...,outlier k=100,cluster k=500,distance k=500,outlier k=500,cluster k=1000,distance k=1000,outlier k=1000,cluster k=1500,distance k=1500,outlier k=1500
243927,243927,19900131,10057,36.0,0,6.120755,0,4,6.181993,0,...,0,19,5.334956,0,103,4.735991e+00,0,1277.0,3.138217e+00,0.0
243928,243928,19900131,10074,34.0,2,7.910803,1,1,7.749312,1,...,1,373,7.251274,1,410,7.150136e+00,1,1097.0,0.000000e+00,0.0
243929,243929,19900131,10082,48.0,3,12.843116,1,5,11.848580,1,...,1,255,8.631617,1,692,0.000000e+00,0,491.0,0.000000e+00,0.0
243930,243930,19900131,10137,49.0,2,4.574791,0,3,3.682299,0,...,0,40,2.403934,0,32,2.559683e+00,0,15.0,2.049353e+00,0.0
243931,243931,19900131,10145,99.0,2,5.942988,0,3,5.659142,0,...,0,53,4.827178,0,456,4.488611e+00,0,880.0,4.355209e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466529,1466529,20201231,93423,79.0,0,5.298085,0,4,5.722224,1,...,0,200,4.177078,0,197,4.400300e+00,0,439.0,3.777693e+00,0.0
1466530,1466530,20201231,93426,36.0,0,4.044636,0,4,4.111018,0,...,0,411,3.405722,0,269,3.737890e+00,0,193.0,3.504675e+00,0.0
1466531,1466531,20201231,93427,36.0,1,4.937171,0,1,4.635658,0,...,0,5,4.303813,0,251,3.133092e+00,0,272.0,3.472513e+00,0.0
1466532,1466532,20201231,93434,1.0,0,9.800538,1,4,9.856951,1,...,1,206,8.013019,1,642,3.371748e-07,0,855.0,3.371748e-07,0.0


In [12]:
#Number of clusters 129
df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(["DATE","cluster k=500"])['permno'].count().pipe(lambda  dfx: dfx.loc[dfx>1]).count()/372
#.count().max()
#['permno'].pipe(lambda  dfx: dfx.loc[dfx>1])
#218.3279569892473

218.3279569892473

In [20]:
#Number of stocks in total 3157
df_1990_2000.groupby(['DATE'])['permno'].count().mean()
#3286.5779569892475

3286.5779569892475

In [34]:
#Number of stocks in clusters 1257 (39.78) - from paper
df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(['DATE','cluster k=500']).count()['permno'].pipe(lambda  dfx: dfx.loc[dfx>1]).sum()/372
#1989.8172043010752

1989.8172043010752

In [15]:
#Number of outliers 1900 (60.22) - from paper
num_outliers_original =  df_1990_2000[df_1990_2000["outlier k=500"] == 1].count()['permno']/372
num_outliers_original_after_cleaned_outiers = df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(['DATE','cluster k=500']).count()['permno'].pipe(lambda  dfx: dfx.loc[dfx==1]).sum()/372
num_outliers_original + num_outliers_original_after_cleaned_outiers
#1296.760752688172

1296.760752688172

In [17]:
num_outliers_original_after_cleaned_outiers

268.6209677419355

In [8]:
#Number of stocks in the biggest cluster 63 (5.18)
df = df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(['DATE','cluster k=500']).count()['permno'].pipe(lambda  dfx: dfx.loc[dfx>1])
df = df.to_frame('number_in_cluster')
mean1st = df.sort_values(['DATE','number_in_cluster'],ascending=False).groupby('DATE').head(1).mean()
mean1st
#67.341398

number_in_cluster    67.341398
dtype: float64

In [9]:
#Number of stocks in the second biggest cluster 51 (4.13)
df = df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(['DATE','cluster k=500']).count()['permno'].pipe(lambda  dfx: dfx.loc[dfx>1])
df = df.to_frame('number_in_cluster')
mean2nd = df.sort_values(['DATE','number_in_cluster'],ascending=False).groupby('DATE').head(2).mean()*2-mean1st
mean2nd
#56.303763

number_in_cluster    56.303763
dtype: float64

In [10]:
#Number of stocks in the third biggest cluster 45 (3.63)
df = df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(['DATE','cluster k=500']).count()['permno'].pipe(lambda  dfx: dfx.loc[dfx>1])
df = df.to_frame('number_in_cluster')
mean3rd = df.sort_values(['DATE','number_in_cluster'],ascending=False).groupby('DATE').head(3).mean()*3-mean1st-mean2nd
mean3rd
#50.680108

number_in_cluster    50.680108
dtype: float64

In [40]:
df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(['DATE','cluster k=500']).count()['permno'].pipe(lambda  dfx: dfx.loc[dfx==1]).sum()/372

268.6209677419355

In [110]:
#Number of clusters 129 - The number of clusters, K, is set to 500, but the outlier removing process eliminates some clusters 
# whose data points are far from each other and the remaining clusters are less than 200.The k-means clustering forms an average of 129 clusters after
#removing outliers, with a range of 80 to 165 clusters
df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(['DATE']).nunique()["cluster k=500"].mean()

486.9489247311828

In [97]:
#Number of stocks in the biggest cluster 63 (5.18)
df_1990_2000[df_1990_2000["outlier k=1500"] == 0].groupby(["DATE","cluster k=1500"])['permno'].count().max()

57

In [101]:
#Number of stocks in the biggest cluster 63 (5.18)
df_1990_2000[df_1990_2000["outlier k=500"] == 0].groupby(["DATE","cluster k=500"])['permno'].count().max()

119